In [1]:
import os
import mujoco

import numpy
numpy.set_printoptions(precision=3, suppress=True, linewidth=100)

import mediapy as media
import matplotlib.pyplot as plt
import math

In [2]:
Vnom = 6
R = Vnom/.600
G = 55.5
t_stall = 15/100/G
i_stall = .6
i_nl = .2
w_nl = .66*1000*2*math.pi/180*G

kt = t_stall/ i_stall
ke = kt

b_calc = kt*i_nl/w_nl
ts = 1e-4

V_control = 5
b_fit = 1.404e-06
kp_fit = 8.896


In [3]:
w_nl


1278.6282100110457

In [4]:
t_stall

0.0027027027027027024

In [5]:
framerate = 30  # (Hz)
data_rate = 100  # (Hz)
width = 800
height = 600

In [6]:
xml_template = """
<mujoco>
    <option><flag gravity="enable" contact="enable" /></option>
    <option timestep="1e-4"/>
    <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <default>
      <geom contype="1" conaffinity="1" condim="3" friction=".6 .3 .3" solimp=".999 .999 .001" solref=".001 1" margin="0.001" group="0"/>
  </default>

    <worldbody>
    <light name="top" pos="0 0 1"/>

    
    <camera />
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 -.05" size="1 1 .05" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" euler="0 0 0" pos="0 0 .05">
        <joint name="joint_1" type="slide" axis="0 0 1" pos="0 0 0"/>
        <geom name="trunk" type="box" size=".025 .025 .025" pos="0 0 0" rgba="1 0 0 1" mass=".01"/>

        <body name="leg_link_1" euler="0 0 0" pos=".025 0 -.025">
            <joint name="joint_2" type="hinge" axis="0 1 0" pos="0 0 0"  limited="true" range="0 180"/>
            <geom type="box" size=".025 .0125 .001" pos=".025 0 0" rgba="1 0 0 1" mass=".001"/>
            <body name="leg_link_2" euler="0 0 0" pos=".05 0 0">
                <joint name="joint_3" type="hinge" axis="0 1 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" />
                <geom type="box" size=".025 .0125 .001" pos=".025 0 0" rgba="1 0 0 1" mass=".001"/>
            </body>
        </body>
    </body>
  </worldbody>
  <actuator>
    <motor name="motor_1" joint="joint_2"/>
  </actuator>
</mujoco>
"""

In [7]:
k = .1
b = .01
render=True


xml = xml_template.format(k=k,b=b,width=width,height=height)
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)


def my_controller(model, data):
    w = data.qvel[1]
    actual = data.qpos[1]

    t = data.time
    if t > 1:
        desired = math.pi
    else:
        desired = 0
    error = desired-actual
    V = kp_fit*error
    if V>V_control: V=V_control
    if V<-V_control: V=-V_control
    torque = (kt*(V-(ke)*w*G)/R-b_fit*w*G)*G

    data.ctrl[0] = torque
    return

mujoco.set_mjcb_control(my_controller)

duration = 5  # (seconds)


frames = []
t = []
xy = []

mujoco.mj_resetData(model, data)
while data.time < duration:
    mujoco.mj_step(model, data)


    if render:

        if len(frames) < data.time * framerate:
            renderer.update_scene(data)
            pixels = renderer.render()
            frames.append(pixels)

    if len(frames) < data.time * data_rate:
        t.append(data.time)
        xy.append(data.xpos.copy())

mujoco.set_mjcb_control(None)


if render:
    media.show_video(frames, fps=framerate,width=width,height=height)
t = numpy.array(t)
xy = numpy.array(xy)


This browser does not support the video tag.
